In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sklearn
import seaborn as sb
import matplotlib.pyplot as plt
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/sl-gene-expression/14cancer.xtrain
/kaggle/input/sl-gene-expression/14cancer.ytrain


# Data Acquisition

### Gene Dataset

In [2]:
gene_train_df = pd.read_csv('/kaggle/input/sl-gene-expression/14cancer.xtrain', header=None, delim_whitespace=True)

In [3]:
gene_train_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,-73,-16,4,-31,-33,-37,-18,-26,-40,22,...,274,-96,-96,-124,-201,-196,34,-56,-245,-26
1,-69,-63,-45,-110,-39,-90,28,-23,-264,-14,...,-915,-221,-458,-664,-259,-369,-81,-818,-235,-1595
2,-48,-97,-112,-20,-45,-75,10,2,-335,-21,...,-303,-119,-134,-361,22,-263,-146,-1338,-127,-2085
3,13,-42,-25,-50,14,-46,30,34,18,26,...,29,243,109,21,140,162,-151,-57,197,-334
4,-86,-91,-85,-115,-56,-45,-56,-54,-163,-42,...,-171,-224,-630,-519,-277,-277,-174,-989,-562,-455
5,-147,-164,-127,-113,-106,-129,-200,-30,-208,-38,...,-217,-366,-369,-412,-373,-615,-121,-796,-714,-354
6,-65,-53,56,-17,73,87,149,-12,138,-60,...,-958,-291,-1407,-39,263,-397,-290,-1466,-1621,-482
7,-71,-77,-110,-40,-34,-49,-87,-25,-113,-39,...,-320,-69,-337,-279,-82,-243,-106,-347,-226,196
8,-32,-17,81,-17,18,50,-2,-6,16,21,...,-166,-39,-294,6,100,70,43,-413,-35,114
9,100,122,41,80,64,3,95,69,-15,64,...,268,4,351,424,346,-167,240,103,-243,45


In [4]:
gene_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16063 entries, 0 to 16062
Columns: 144 entries, 0 to 143
dtypes: float64(1), int64(143)
memory usage: 17.6 MB


In [5]:
gene_train_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
count,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,...,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000
mean,26.599950,23.171388,81.767540,31.268879,167.069974,74.395816,82.844923,33.377825,264.759260,31.306792,...,638.439706,354.352798,912.274731,378.876860,259.743946,458.837764,160.001245,598.945776,293.960095,164.547158
std,251.664362,228.725250,469.891508,233.358432,611.572681,443.675660,468.294695,187.341336,962.057611,265.719834,...,1462.490383,1046.445591,2008.482183,1139.403996,900.029931,1175.377003,588.505841,1499.979802,914.750927,747.615951
min,-3830.000000,-4327.000000,-2375.000000,-2133.000000,-2778.000000,-5524.000000,-3943.000000,-909.000000,-3895.000000,-1265.000000,...,-4513.000000,-11743.000000,-6873.000000,-9655.000000,-7405.000000,-11261.000000,-3480.000000,-6818.000000,-7479.000000,-6978.000000
25%,-38.000000,-29.000000,-19.000000,-18.000000,5.000000,-9.000000,-16.000000,-5.000000,-4.000000,-9.000000,...,23.000000,9.000000,11.000000,-8.000000,-14.000000,2.000000,-7.000000,-1.000000,-12.000000,-22.000000
50%,8.000000,10.000000,19.000000,13.000000,47.000000,19.000000,20.000000,11.000000,63.000000,8.000000,...,202.000000,126.000000,206.000000,108.000000,83.000000,158.000000,47.000000,150.000000,91.000000,41.000000
75%,65.000000,57.000000,75.000000,49.000000,147.000000,62.000000,72.000000,33.000000,236.000000,33.000000,...,734.000000,394.000000,1038.000000,417.000000,291.000000,556.000000,171.000000,701.000000,349.000000,190.000000
max,12577.000000,13111.000000,17258.000000,16144.000000,14917.000000,17061.000000,16766.000000,12874.000000,24741.000000,17861.000000,...,30117.000000,19868.000000,25205.000000,21274.000000,16444.000000,19158.000000,18270.000000,21524.000000,19556.000000,18310.000000


### Cancer categories dataset

In [6]:
cancer_categories_df = pd.read_csv('/kaggle/input/sl-gene-expression/14cancer.ytrain', header=None, delim_whitespace=True)

In [7]:
cancer_categories_df.head()

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,1,1,1,1,1,1,1,1,2,2,...,14,14,14,14,14,14,14,14,14,14


# Data Cleaning

### Rename columns

In [8]:
# Gene dataset & Cancer categories dataset
new_columns = []
for i in range(1,len(gene_train_df.columns) + 1, 1):
    new_columns.append(f"Patient{i}")
gene_train_df.columns = new_columns
cancer_categories_df.columns = new_columns

# Data Enrichment

# Final Dataset

### Gene dataset

In [9]:
gene_train_df.head()

,Patient1,Patient2,Patient3,Patient4,Patient5,Patient6,Patient7,Patient8,Patient9,Patient10,...,Patient135,Patient136,Patient137,Patient138,Patient139,Patient140,Patient141,Patient142,Patient143,Patient144
0,-73,-16,4,-31,-33,-37,-18,-26,-40,22,...,274,-96,-96,-124,-201,-196,34,-56,-245,-26
1,-69,-63,-45,-110,-39,-90,28,-23,-264,-14,...,-915,-221,-458,-664,-259,-369,-81,-818,-235,-1595
2,-48,-97,-112,-20,-45,-75,10,2,-335,-21,...,-303,-119,-134,-361,22,-263,-146,-1338,-127,-2085
3,13,-42,-25,-50,14,-46,30,34,18,26,...,29,243,109,21,140,162,-151,-57,197,-334
4,-86,-91,-85,-115,-56,-45,-56,-54,-163,-42,...,-171,-224,-630,-519,-277,-277,-174,-989,-562,-455


In [10]:
gene_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16063 entries, 0 to 16062
Columns: 144 entries, Patient1 to Patient144
dtypes: float64(1), int64(143)
memory usage: 17.6 MB


In [11]:
gene_train_df.describe()

,Patient1,Patient2,Patient3,Patient4,Patient5,Patient6,Patient7,Patient8,Patient9,Patient10,...,Patient135,Patient136,Patient137,Patient138,Patient139,Patient140,Patient141,Patient142,Patient143,Patient144
count,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,...,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000,16063.000000
mean,26.599950,23.171388,81.767540,31.268879,167.069974,74.395816,82.844923,33.377825,264.759260,31.306792,...,638.439706,354.352798,912.274731,378.876860,259.743946,458.837764,160.001245,598.945776,293.960095,164.547158
std,251.664362,228.725250,469.891508,233.358432,611.572681,443.675660,468.294695,187.341336,962.057611,265.719834,...,1462.490383,1046.445591,2008.482183,1139.403996,900.029931,1175.377003,588.505841,1499.979802,914.750927,747.615951
min,-3830.000000,-4327.000000,-2375.000000,-2133.000000,-2778.000000,-5524.000000,-3943.000000,-909.000000,-3895.000000,-1265.000000,...,-4513.000000,-11743.000000,-6873.000000,-9655.000000,-7405.000000,-11261.000000,-3480.000000,-6818.000000,-7479.000000,-6978.000000
25%,-38.000000,-29.000000,-19.000000,-18.000000,5.000000,-9.000000,-16.000000,-5.000000,-4.000000,-9.000000,...,23.000000,9.000000,11.000000,-8.000000,-14.000000,2.000000,-7.000000,-1.000000,-12.000000,-22.000000
50%,8.000000,10.000000,19.000000,13.000000,47.000000,19.000000,20.000000,11.000000,63.000000,8.000000,...,202.000000,126.000000,206.000000,108.000000,83.000000,158.000000,47.000000,150.000000,91.000000,41.000000
75%,65.000000,57.000000,75.000000,49.000000,147.000000,62.000000,72.000000,33.000000,236.000000,33.000000,...,734.000000,394.000000,1038.000000,417.000000,291.000000,556.000000,171.000000,701.000000,349.000000,190.000000
max,12577.000000,13111.000000,17258.000000,16144.000000,14917.000000,17061.000000,16766.000000,12874.000000,24741.000000,17861.000000,...,30117.000000,19868.000000,25205.000000,21274.000000,16444.000000,19158.000000,18270.000000,21524.000000,19556.000000,18310.000000


### Cancer categories dataset

In [12]:
cancer_categories_df.head(10)

,Patient1,Patient2,Patient3,Patient4,Patient5,Patient6,Patient7,Patient8,Patient9,Patient10,...,Patient135,Patient136,Patient137,Patient138,Patient139,Patient140,Patient141,Patient142,Patient143,Patient144
0,1,1,1,1,1,1,1,1,2,2,...,14,14,14,14,14,14,14,14,14,14


In [13]:
cancer_categories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 144 entries, Patient1 to Patient144
dtypes: int64(144)
memory usage: 1.2 KB


In [14]:
cancer_categories_df.describe()

,Patient1,Patient2,Patient3,Patient4,Patient5,Patient6,Patient7,Patient8,Patient9,Patient10,...,Patient135,Patient136,Patient137,Patient138,Patient139,Patient140,Patient141,Patient142,Patient143,Patient144
count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
